Query on Cosmohub to get Gaia EDR3 data:

    SELECT `solution_id`, `designation`, `source_id`, `ra`, `ra_error`, `dec`, `dec_error`, `parallax`, `parallax_error`, `pm`, `pmra`, `pmra_error`, `pmdec`, `pmdec_error`, `ra_dec_corr`, `ra_parallax_corr`, `ra_pmra_corr`, `ra_pmdec_corr`, `dec_parallax_corr`, `dec_pmra_corr`, `dec_pmdec_corr`, `parallax_pmra_corr`, `parallax_pmdec_corr`, `pmra_pmdec_corr`, `visibility_periods_used`, `astrometric_sigma5d_max`, `ruwe`, `duplicated_source`, `phot_g_mean_flux`, `phot_g_mean_flux_error`, `phot_g_mean_flux_over_error`, `phot_g_mean_mag`, `phot_bp_mean_flux`, `phot_bp_mean_flux_error`, `phot_bp_mean_flux_over_error`, `phot_bp_mean_mag`, `phot_rp_mean_flux`, `phot_rp_mean_flux_error`, `phot_rp_mean_flux_over_error`, `phot_rp_mean_mag`, `dr2_radial_velocity`, `dr2_radial_velocity_error`, `l`, `b`, `phot_bp_rp_excess_factor` FROM gaia_edr3 WHERE `phot_g_mean_mag` < 18.5 AND (`b` > 30 OR `b` < -30)

In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from pyia import GaiaData

In [ ]:
g = GaiaData('/Users/apricewhelan/data/GaiaEDR3/edr3-rv-good-plx-result.fits')

In [ ]:
b_mask = np.abs(g.b) > 30*u.deg

In [ ]:
# Raw photometry
# MG = g.phot_g_mean_mag - g.distmod
# BPRP = g.bp_rp

# attempt at extinction-correcting the photometry:
MG = g.get_G0() - g.distmod
BPRP = g.get_BP0() - g.get_RP0()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5.5),
                         sharex=True, sharey=True)
bins = (np.linspace(-1, 4, 256),
        np.linspace(-6, 12, 256))

axes[0].hist2d(
    BPRP.value, 
    MG.value,
    bins=bins,
    norm=mpl.colors.LogNorm()
)

axes[1].hist2d(
    BPRP.value[b_mask], 
    MG.value[b_mask],
    bins=bins,
    norm=mpl.colors.LogNorm()
)

axes[0].set_ylim(12, -6)
for ax in axes:
    ax.set_xlabel(r'$G_{\rm BP}-G_{\rm RP}$')
axes[0].set_ylabel('$M_G$')

fig.tight_layout()

Select the upper main sequence (but not too upper, and remove the giant branch). These $M_G$ limits correspond roughly to F-type stars through G stars based on:
https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt

In [ ]:
MG_lim = (2.5, 5.5)
max_G = 18.5
maxdist = coord.Distance(distmod=max_G - max(MG_lim))
print(f"rough maximum distance for completeness: {maxdist:.0f}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.hist2d(
    BPRP.value[b_mask], 
    MG.value[b_mask],
    bins=(np.linspace(0, 2.3, 256),
          np.linspace(0, 8, 256)),
    norm=mpl.colors.LogNorm()
)

nodes = np.array([
    [1.1, MG_lim[1]],
    [0.6, MG_lim[1]],
    [0.2, MG_lim[0]],
    [0.8, MG_lim[0]],
    [0.93, 4.5],
    [1.1, MG_lim[1]]
])
ax.plot(nodes[:, 0], nodes[:, 1], color='tab:green')
path_ = mpl.path.Path(nodes)
cmd_mask = path_.contains_points(np.stack((BPRP.value, MG.value)).T)

ax.set_ylim(8, 0)
ax.set_xlabel(r'$G_{\rm BP}-G_{\rm RP}$')
ax.set_ylabel('$M_G$')
fig.tight_layout()

In [ ]:
mask = (
    b_mask &
    cmd_mask
)
b_mask.sum(), cmd_mask.sum(), mask.sum()

In [ ]:
ms_g = g[mask]

In [ ]:
gal = ms_g.skycoord.galactic
x, y, z = gal.represent_as('cartesian').xyz.to_value(u.kpc)

In [ ]:
fig, axes = plt.subplots(
    1, 2, 
    figsize=(12, 6), 
    constrained_layout=True
)

for ax, coords, labels in zip(axes, 
                              [(x, y), (x, z)], 
                              [('x', 'y'), ('x', 'z')]):
    ax.hist2d(
        coords[0], 
        coords[1], 
        bins=np.linspace(-0.5, 0.5, 128), 
        cmap='Greys'
    #     norm=mpl.colors.LogNorm()
    )

    circ = mpl.patches.Circle(
        (0, 0),
        radius=maxdist.to_value(u.kpc),
        linewidth=2, 
        facecolor='none',
        edgecolor='tab:red'
    )
    ax.add_patch(circ)
    ax.set_aspect('equal')

    ax.set(
        xlabel=f'Heliocentric Galactic ${labels[0]}$ [kpc]',
        ylabel=f'Heliocentric Galactic ${labels[1]}$ [kpc]',
        xlim=(-0.5, 0.5),
        ylim=(-0.5, 0.5),
        xticks=np.arange(-0.5, 0.5+0.1, 0.2),
        yticks=np.arange(-0.5, 0.5+0.1, 0.2)
    );

In [ ]:
plt.hist(z, bins=np.linspace(-1, 1, 256));

Parameters:
- $z_\odot$
- orientation of the galactic plane

These cartesian coordinates are a rotation $\mathbf{M}$ away from the ICRS cartesian coordinates
$$
\textbf{x} = (x, y, z) = \mathbf{M}\,\textbf{x}_{\textrm{ICRS}}
$$
where one angle in the rotation matrix is set by the (fixed) sky position of the Galactic center, one is set by the Sun's height above the midplane and distance to the Galactic center, and one is set by the orientation of the Galactic plane (on the sky?) or position angle or "roll" or whatever.

In the rotated "Galactocentric" coordinates $\textbf{x}$:
$$
\rho(\textbf{x}) = \rho_0 \, \left[
    \textrm{sech}^2\left(\frac{z + z_\odot}{2\,h_1}\right) + 
    \alpha \, \textrm{sech}^2\left(\frac{z + z_\odot}{2\,h_2}\right)
    \right]
$$
where the density is assumed to be uniform in $x, y$ and only depend on $z$.